# ETL-pipelines in Azure Databricks
## Module 7: Best Practices en Security
In deze module leren we hoe we ETL-processen kunnen **optimaliseren, beveiligen en foutafhandeling** implementeren.

## Stap 1: Partitioning en Z-Ordering
Partitioning en Z-Ordering verbeteren de prestaties van queries.

In [ ]:
df_partitioned = df_declaraties.repartition("Zorgtype")
df_partitioned.write.format("delta").mode("overwrite").partitionBy("Zorgtype").save("/mnt/data/zilver/declaraties_partitioned")

spark.sql("OPTIMIZE delta.`/mnt/data/zilver/declaraties_partitioned` ZORDER BY (Declaratie_Datum)")

## Stap 2: Role-Based Access Control (RBAC)
We beheren de toegang tot data met RBAC.

In [ ]:
spark.sql("GRANT SELECT ON delta.`/mnt/data/zilver/declaraties` TO `data_analyst`;")
spark.sql("GRANT MODIFY ON delta.`/mnt/data/zilver/declaraties` TO `data_engineer`;")

## Stap 3: Gevoelige Data Maskeren
We maskeren gevoelige NAW-gegevens.

In [ ]:
from pyspark.sql.functions import expr

df_masked = df_verzekerden.withColumn("Naam", expr("concat(substr(Naam, 1, 1), '****')"))
df_masked.show(5)

## Stap 4: Logging en Foutafhandeling
We loggen fouten in een Delta Lake tabel.

In [ ]:
from pyspark.sql.functions import lit
import datetime

def log_error(error_message):
    error_df = spark.createDataFrame(
        [(str(datetime.datetime.now()), error_message)], 
        ["Timestamp", "Error"]
    )
    error_df.write.format("delta").mode("append").save("/mnt/data/logs/errors")

# Simuleer een fout
log_error("Testfout: Geen toegang tot Delta Lake tabel")

## Stap 5: Optimaliseren van Delta Lake
We verbeteren prestaties met OPTIMIZE en VACUUM.

In [ ]:
spark.sql("OPTIMIZE delta.`/mnt/data/zilver/declaraties`")
spark.sql("VACUUM delta.`/mnt/data/zilver/declaraties` RETAIN 7 DAYS")

## Oefeningen
1. **Partitioneer de verzekerden dataset op Regio en pas Z-Ordering toe op Leeftijdscategorie.**
2. **Stel RBAC in voor de verzekerden dataset.**
3. **Mask de geboortedatum van verzekerden.**
4. **Maak een Delta-tabel met gelogde fouten en query deze met Spark SQL.**
5. **Maak een governance-dashboard in Databricks voor data lineage.**